# library

In [2]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import random

2022-06-10 20:30:25.956910: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


# 3X 1y data

In [2]:
d1215 = pd.read_feather('d1215.ftr', columns = None, use_threads = True)

In [3]:
d1215.shape

(1084047, 823)

In [4]:
a1215 = d1215.to_numpy()

In [6]:
len_a2012 = 284412
len_a2013 = 242826
len_a2014 = 242448
len_a2015 = 314361
len_a2016 = 263593
len_a2017 = 216591
len_a2018 = 307395
len_a2019 = 278610
len_a2020 = 371006
len_a2021 = 321808

# df로 X,y train test

In [6]:
# d1215.shape[0] == d1215.iloc[:-len_a2015,:-1].shape[0] + d1215.iloc[-len_a2015:,:-1].shape[0]
X_train = d1215.iloc[:-len_a2015,2:-1]
y_train = d1215.iloc[:-len_a2015,-1]
X_test = d1215.iloc[-len_a2015:,2:-1]
y_test = d1215.iloc[-len_a2015:,-1]

# 한 번에

In [7]:
results = []
def my_regressor():
#     X = d1215.iloc[:-len_a2015,2:-1] # 1214 데이터
#     y = d1215.iloc[:-len_a2015,-1]
    X = d1215.iloc[:,2:-1] # 1215 데이터
    y = d1215.iloc[:,-1]
    from sklearn.linear_model import LinearRegression # 일반 회귀 모델
    from sklearn.linear_model import Ridge, Lasso, ElasticNet # Norm 규제 회귀 모델
    from sklearn.linear_model import ARDRegression, BayesianRidge # 베이지안 회귀
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor # decision tree 앙상블 모델, 배깅
    from xgboost import XGBRegressor # decision tree 앙상블 모델, 부스팅
    import lightgbm as lgb
    from sklearn.model_selection import GridSearchCV
#     from sklearn.model_selection import cross_val_score # model 검증
    
#     from sklearn.model_selection import StratifiedShuffleSplit
    from sklearn.model_selection import ShuffleSplit
    from sklearn.model_selection import cross_val_score
    
    from sklearn.dummy import DummyRegressor
    from sklearn.metrics import mean_squared_error as mse # 모델 평가 지표 scoring (mse)
    from sklearn.metrics import r2_score as r2
    
    import ast # convert string to function
    
    linear = LinearRegression()
    ridge, lasso, elasticnet = Ridge(), Lasso(), ElasticNet()
    ardr_linear, baysian_ridge = ARDRegression(), BayesianRidge()
    DecisionTree = DecisionTreeRegressor(max_depth = 10)
    random_forest = RandomForestRegressor(n_estimators=10,
                                          n_jobs=16,
                                          max_depth=10,
                                          random_state=2,
                                          verbose=2,
                                          max_leaf_nodes=4)
    xgboost_linear = XGBRegressor()
    gbm = lgb.LGBMRegressor(num_leaves=31,
                            learning_rate=0.05,
                            n_estimators=20)

    dummy = DummyRegressor(strategy = 'mean')
    
    my_model_list = ['linear', 'ridge', 'lasso', 'elasticnet',
                     'ardr_linear', 'baysian_ridge',
                     'xgboost_linear', 'gbm', 'dummy',
                     'DecisionTree', 'random_forest'] # 오래 걸리는건 뒤로 뺌
    
    # seed 고정
    user_seed = 0
    random.seed(user_seed) # seed 고정
    
    # train끼리 idx 같아야 하므로
    len_train = len_a2012+len_a2013+len_a2014
    train_idx = list(range(len_train))
    random.shuffle(train_idx) # 자동으로 덮어쓰기
    
    len_test = len_train + len_a2015
    test_idx = list(range(len_train,len_test)) # +1 안해줘도 되는 거 위 cell에서 확인
    random.shuffle(test_idx) # 자동으로 덮어쓰기
    
    i = 0
    ######################################################################### fitting
    for model_nm in tqdm(my_model_list):
        ############################################################
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_test = X.iloc[test_idx]
        y_test = y.iloc[test_idx]
#         X_train = (X.iloc[:-len_a2015,:]).sample(frac=1, random_state = user_seed)
#         y_train = (y.iloc[:-len_a2015,:]).sample(frac=1, random_state = user_seed)
#         X_test = (X.iloc[-len_a2015:,:]).sample(frac=1, random_state = user_seed)
#         y_test = (y.iloc[-len_a2015:,:]).sample(frac=1, random_state = user_seed)
        ############################################################
        y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
        y_test = np.array(y_test).reshape(-1,1)

        ########################################################

        # fitting
        if model_nm == 'gbm':
            model_ = eval(model_nm).fit(X_train, y_train,
                                        eval_set=[(X_test, y_test)],
                                        eval_metric='l1',
                                        early_stopping_rounds=5)
            y_pred_test = model_.predict(X_test, num_iteration=gbm.best_iteration_) # 예측

        else:
            model_ = eval(model_nm).fit(X_train, y_train)
            y_pred_test = model_.predict(X_test) # 예측

        mse_score = mse(y_test, y_pred_test)
        r2_score = r2(y_test, y_pred_test)
        # cv_scores = cross_val_score(eval(model_nm), X, y, cv=ss)

        results.append([i, f'{model_nm}', mse_score, r2_score]) #[:str(model).index("(")]
        i+=1
        
#         # cv (안함)
#         ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
#         for train_idx, test_idx in tqdm(ss.split(X,y)):
#             x_train = d1215.iloc[train_idx,2:-1] # 식별변수 제외 나머지 열 ~ NC열 전까지
#             y_train = d1215.iloc[train_idx,-1]
#             x_test = d1215.iloc[test_idx,2:-1] # 식별변수 제외 나머지 열 ~ NC열 전까지
#             y_test = d1215.iloc[test_idx,-1]
            
            
#             y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
#             y_test = np.array(y_test).reshape(-1,1)

#             ########################################################
        
#             # fitting
#             if model_nm == 'gbm':
#                 model_ = eval(model_nm).fit(X_train, y_train,
#                                             eval_set=[(X_test, y_test)],
#                                             eval_metric='l1',
#                                             early_stopping_rounds=5)
#                 y_pred_test = model_.predict(x_test, num_iteration=gbm.best_iteration_) # 예측
                
#             else:
#                 model_ = eval(model_nm).fit(x_train, y_train)
#                 y_pred_test = model_.predict(x_test) # 예측

#             mse_score = mse(y_test, y_pred_test)
#             r2_score = r2(y_test, y_pred_test)
#             # cv_scores = cross_val_score(eval(model_nm), X, y, cv=ss)

#             results.append([i, f'{model_nm}', mse_score, r2_score]) #[:str(model).index("(")]
#             i+=1
#             #####################################################################################
        print(results[-5:])
    return results#score_dic#[0]

In [ ]:
import time
start = time.time()
_ = my_regressor()
results
time.time()-start

  0%|                                                    | 0/11 [00:00<?, ?it/s]

Tree기반 너무 오래 걸림... max_depth, n_jobs 같은거 설정해야할듯 <br>
00:23 <br>
01:10 <br>
02:00 <br>
02:45 <br>

[random forest hyperparameter tuning](https://velog.io/@emseoyk/%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D) <br>

- 지금은 1214로 train test shuffle split해서 모델링 하는 중.
- 만약 15를 test로 하고싶다면 shuffle split은 포기해야함.
- 그렇다면 df자체적으로 뒤섞는걸 하면 되려나?
- 학습 순서가 의미 있을지도 모르니까. (전에 학습하는 순서 의미 있다는걸 듣긴 했음.)

- df row 순서 섞는거는 0(처음)부터 2014년도의 마지막 데이터가 있는 행까지를 섞어서 인덱스를 다시 붙여주면 됨. (뭔 소리 하고있는건지 모르겠지만 아무튼 그렇다...)

- df.sample(frac=1, random_state = 1)로 진행

In [ ]:
results

# plot을 위한 준비

In [ ]:
dic = {}
for i in results:
    if i[1] not in dic: # dic.keys()라고 써도 됨.
        dic[i[1]] = [i[2]]
    else:
        dic[i[1]].append(i[2])


# import math
# 리스트에 mean 함수가 내장이 안돼있다니...

for i in dic.keys():
    dic[i] = sum(dic[i])/len(dic[i])


score_lst = sorted(dic.items(), key = lambda t : t[1]) #mse 작은 순으로 줄 세워짐.
score_lst
# dic.keys()

In [ ]:
score_dic = {}
for i in score_lst:
    score_dic[i[0]] = i[1]

In [ ]:
df = pd.DataFrame(score_dic.items(), columns=['key', 'value'])
df

In [ ]:
df[df.key=='dummy']['value'].values
f'dummy\n: {dummy_mean[0]:.2f}'

# MSE plot

In [ ]:
g = sns.barplot(x = 'key', y = 'value', data=df, capsize=.2)
_ = g.set_xticks(range(len(df)))
_ = g.set_xticklabels(g.get_xticklabels(), rotation=45)
_ = g.set_xlabel("regression models", fontsize = 15)
_ = g.set_ylabel("Mean Squared Errors", fontsize = 15)
_ = g.set_ylim([0, 0.0155]) # ax.set(ylim=(0.4, 0.6))
dummy_mean = df[df.key=='dummy']['value'].values
_ = g.plot([-1,len(df)], [dummy_mean, dummy_mean], color = 'red')
_ = g.annotate(text = f'dummy\n: {dummy_mean[0]:.4f}', xy = (3, dummy_mean), xytext = (0.1, 0.0115),
               arrowprops = dict(facecolor = 'red', shrink = 0),)


val = df['value']
cnt = 0
for i in range(len(df)):
    _ = g.text(cnt-0.27, 0.0007, f'{val[i]:.2f}')
    cnt += 1

plt.tight_layout()

# ROC - AUC plot..? 은 분류에서...
[ROC-AUC 공식문서](https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score